This file is **very similar** to Preliminary-analysis-1.

In [137]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [138]:
data = pd.read_csv('consolidated-1.csv')

In [139]:
time_window = '3min' # This is the time window over which we will average call arrivals.
time_window_size = int(time_window[0])

In [140]:
# This is the time in seconds taken to complete a call(included post-processing).
data['Avg-time'] = data['Avg-time'].fillna(0) 

In [141]:
data['lambda'] = data['ncalls'].rolling(window=5).mean() # Number of calls per 3 minutes.
# The rolling window size has nothing to do with the time window.

In [142]:
data.set_index('Create-time', inplace=True)

In [143]:
data['mu'] = data['Avg-time']/(time_window_size * 60) # The term lambda/mu must be unit less.

In [144]:
data['E'] = data['lambda']*data['mu'] # Ordered traffic in erlangs

In [154]:
def prob_of_wait(E, m):
    """
    E:      traffic
    m:      # agents.
    Reference: https://en.wikipedia.org/wiki/Erlang_(unit)
    """ 
    p = 1
               
    if m > E:   
        try:
            numerator = E**m/math.factorial(m) * m/(m - E)
            denominator = 0

            for i in range(m):
                denominator += E**i/math.factorial(i)

            denominator += numerator

            p = numerator/denominator
        except OverflowError:
            print(f'Overflow due to E = {E}, m = {m}')
        
    return p

def calculate_ASA(wait_prob, mu, nagents, E):
    a_large_number = 1000
    
    if nagents > E:
        return wait_prob * mu/(nagents - E) * time_window_size
    else:
        return a_large_number
    
def find_nagents(E, m):
    start = m

    if math.isnan(E):
        return start
    
    if E > m:
        threshold = 0.8
        start = int(E)
        while (prob_of_wait(E, start) > threshold) and (start <= E):
            start += 1            
        
    return start

def find_nagents_1(E, m):
    start = m
             
    if not math.isnan(E):
        threshold = 0.8
        while prob_of_wait(E, start) > threshold:
            start += 1
            
    return start


In [146]:
data['wait-prob'] = data.apply(lambda r: prob_of_wait(r['E'], r['nagents']), axis=1)

Refer to [point 15](https://www.callcentrehelper.com/erlang-c-formula-example-121281.htm) for the formula to compute ASA

In [147]:
data['asa'] = data.apply(lambda r: calculate_ASA(r['wait-prob'], r['mu'], r['nagents'], r['E']), axis=1)

In [148]:
data.head()

,ncalls,Avg-time,Period,nagents,office-hour,Avg-calls,iso_day_of_week,lambda,mu,E,wait-prob,asa
Create-time,,,,,,,,,,,,
2020-09-21 09:06:00,1,79.00,2020-09-21,22,True,NaN,1,NaN,0.438889,NaN,1.0,1000.0
2020-09-21 09:09:00,2,1161.00,2020-09-21,22,True,NaN,1,NaN,6.450000,NaN,1.0,1000.0
2020-09-21 09:12:00,20,452.65,2020-09-21,22,True,7.666667,1,NaN,2.514722,NaN,1.0,1000.0
2020-09-21 09:15:00,20,612.70,2020-09-21,22,True,14.000000,1,NaN,3.403889,NaN,1.0,1000.0
2020-09-21 09:18:00,12,536.50,2020-09-21,22,True,17.333333,1,11.0,2.980556,32.786111,1.0,1000.0


In [156]:
find_nagents(148, 149)

149

In [155]:
data['nagents_reqd'] = data.apply(lambda r: find_nagents_1(r['E'], r['nagents']), axis = 1)

Overflow due to E = 148.3022222222222, m = 149
Overflow due to E = 148.3022222222222, m = 150
Overflow due to E = 148.3022222222222, m = 151
Overflow due to E = 148.3022222222222, m = 152
Overflow due to E = 148.3022222222222, m = 153
Overflow due to E = 148.3022222222222, m = 154
Overflow due to E = 148.3022222222222, m = 155
Overflow due to E = 148.3022222222222, m = 156
Overflow due to E = 148.3022222222222, m = 157
Overflow due to E = 148.3022222222222, m = 158
Overflow due to E = 148.3022222222222, m = 159
Overflow due to E = 148.3022222222222, m = 160
Overflow due to E = 148.3022222222222, m = 161
Overflow due to E = 148.3022222222222, m = 162
Overflow due to E = 148.3022222222222, m = 163
Overflow due to E = 148.3022222222222, m = 164
Overflow due to E = 148.3022222222222, m = 165
Overflow due to E = 148.3022222222222, m = 166
Overflow due to E = 148.3022222222222, m = 167
Overflow due to E = 148.3022222222222, m = 168
Overflow due to E = 148.3022222222222, m = 169
Overflow due 

Overflow due to E = 148.3022222222222, m = 1022
Overflow due to E = 148.3022222222222, m = 1023
Overflow due to E = 148.3022222222222, m = 1024
Overflow due to E = 148.3022222222222, m = 1025
Overflow due to E = 148.3022222222222, m = 1026
Overflow due to E = 148.3022222222222, m = 1027
Overflow due to E = 148.3022222222222, m = 1028
Overflow due to E = 148.3022222222222, m = 1029
Overflow due to E = 148.3022222222222, m = 1030
Overflow due to E = 148.3022222222222, m = 1031
Overflow due to E = 148.3022222222222, m = 1032
Overflow due to E = 148.3022222222222, m = 1033
Overflow due to E = 148.3022222222222, m = 1034
Overflow due to E = 148.3022222222222, m = 1035
Overflow due to E = 148.3022222222222, m = 1036
Overflow due to E = 148.3022222222222, m = 1037
Overflow due to E = 148.3022222222222, m = 1038
Overflow due to E = 148.3022222222222, m = 1039
Overflow due to E = 148.3022222222222, m = 1040
Overflow due to E = 148.3022222222222, m = 1041
Overflow due to E = 148.3022222222222, m

Overflow due to E = 148.3022222222222, m = 1835
Overflow due to E = 148.3022222222222, m = 1836
Overflow due to E = 148.3022222222222, m = 1837
Overflow due to E = 148.3022222222222, m = 1838
Overflow due to E = 148.3022222222222, m = 1839
Overflow due to E = 148.3022222222222, m = 1840
Overflow due to E = 148.3022222222222, m = 1841
Overflow due to E = 148.3022222222222, m = 1842
Overflow due to E = 148.3022222222222, m = 1843
Overflow due to E = 148.3022222222222, m = 1844
Overflow due to E = 148.3022222222222, m = 1845
Overflow due to E = 148.3022222222222, m = 1846
Overflow due to E = 148.3022222222222, m = 1847
Overflow due to E = 148.3022222222222, m = 1848
Overflow due to E = 148.3022222222222, m = 1849
Overflow due to E = 148.3022222222222, m = 1850
Overflow due to E = 148.3022222222222, m = 1851
Overflow due to E = 148.3022222222222, m = 1852
Overflow due to E = 148.3022222222222, m = 1853
Overflow due to E = 148.3022222222222, m = 1854
Overflow due to E = 148.3022222222222, m

Overflow due to E = 148.3022222222222, m = 2648
Overflow due to E = 148.3022222222222, m = 2649
Overflow due to E = 148.3022222222222, m = 2650
Overflow due to E = 148.3022222222222, m = 2651
Overflow due to E = 148.3022222222222, m = 2652
Overflow due to E = 148.3022222222222, m = 2653
Overflow due to E = 148.3022222222222, m = 2654
Overflow due to E = 148.3022222222222, m = 2655
Overflow due to E = 148.3022222222222, m = 2656
Overflow due to E = 148.3022222222222, m = 2657
Overflow due to E = 148.3022222222222, m = 2658
Overflow due to E = 148.3022222222222, m = 2659
Overflow due to E = 148.3022222222222, m = 2660
Overflow due to E = 148.3022222222222, m = 2661
Overflow due to E = 148.3022222222222, m = 2662
Overflow due to E = 148.3022222222222, m = 2663
Overflow due to E = 148.3022222222222, m = 2664
Overflow due to E = 148.3022222222222, m = 2665
Overflow due to E = 148.3022222222222, m = 2666
Overflow due to E = 148.3022222222222, m = 2667
Overflow due to E = 148.3022222222222, m

Overflow due to E = 148.3022222222222, m = 3453
Overflow due to E = 148.3022222222222, m = 3454
Overflow due to E = 148.3022222222222, m = 3455
Overflow due to E = 148.3022222222222, m = 3456
Overflow due to E = 148.3022222222222, m = 3457
Overflow due to E = 148.3022222222222, m = 3458
Overflow due to E = 148.3022222222222, m = 3459
Overflow due to E = 148.3022222222222, m = 3460
Overflow due to E = 148.3022222222222, m = 3461
Overflow due to E = 148.3022222222222, m = 3462
Overflow due to E = 148.3022222222222, m = 3463
Overflow due to E = 148.3022222222222, m = 3464
Overflow due to E = 148.3022222222222, m = 3465
Overflow due to E = 148.3022222222222, m = 3466
Overflow due to E = 148.3022222222222, m = 3467
Overflow due to E = 148.3022222222222, m = 3468
Overflow due to E = 148.3022222222222, m = 3469
Overflow due to E = 148.3022222222222, m = 3470
Overflow due to E = 148.3022222222222, m = 3471
Overflow due to E = 148.3022222222222, m = 3472
Overflow due to E = 148.3022222222222, m

...

Overflow due to E = 148.3022222222222, m = 13224
Overflow due to E = 148.3022222222222, m = 13225
Overflow due to E = 148.3022222222222, m = 13226
Overflow due to E = 148.3022222222222, m = 13227
Overflow due to E = 148.3022222222222, m = 13228
Overflow due to E = 148.3022222222222, m = 13229
Overflow due to E = 148.3022222222222, m = 13230
Overflow due to E = 148.3022222222222, m = 13231
Overflow due to E = 148.3022222222222, m = 13232
Overflow due to E = 148.3022222222222, m = 13233
Overflow due to E = 148.3022222222222, m = 13234
Overflow due to E = 148.3022222222222, m = 13235
Overflow due to E = 148.3022222222222, m = 13236
Overflow due to E = 148.3022222222222, m = 13237
Overflow due to E = 148.3022222222222, m = 13238
Overflow due to E = 148.3022222222222, m = 13239
Overflow due to E = 148.3022222222222, m = 13240
Overflow due to E = 148.3022222222222, m = 13241
Overflow due to E = 148.3022222222222, m = 13242
Overflow due to E = 148.3022222222222, m = 13243
Overflow due to E = 

Overflow due to E = 148.3022222222222, m = 14070
Overflow due to E = 148.3022222222222, m = 14071
Overflow due to E = 148.3022222222222, m = 14072
Overflow due to E = 148.3022222222222, m = 14073
Overflow due to E = 148.3022222222222, m = 14074
Overflow due to E = 148.3022222222222, m = 14075
Overflow due to E = 148.3022222222222, m = 14076
Overflow due to E = 148.3022222222222, m = 14077
Overflow due to E = 148.3022222222222, m = 14078
Overflow due to E = 148.3022222222222, m = 14079
Overflow due to E = 148.3022222222222, m = 14080
Overflow due to E = 148.3022222222222, m = 14081
Overflow due to E = 148.3022222222222, m = 14082
Overflow due to E = 148.3022222222222, m = 14083
Overflow due to E = 148.3022222222222, m = 14084
Overflow due to E = 148.3022222222222, m = 14085
Overflow due to E = 148.3022222222222, m = 14086
Overflow due to E = 148.3022222222222, m = 14087
Overflow due to E = 148.3022222222222, m = 14088
Overflow due to E = 148.3022222222222, m = 14089
Overflow due to E = 

Overflow due to E = 148.3022222222222, m = 14829
Overflow due to E = 148.3022222222222, m = 14830
Overflow due to E = 148.3022222222222, m = 14831
Overflow due to E = 148.3022222222222, m = 14832
Overflow due to E = 148.3022222222222, m = 14833
Overflow due to E = 148.3022222222222, m = 14834
Overflow due to E = 148.3022222222222, m = 14835
Overflow due to E = 148.3022222222222, m = 14836
Overflow due to E = 148.3022222222222, m = 14837
Overflow due to E = 148.3022222222222, m = 14838
Overflow due to E = 148.3022222222222, m = 14839
Overflow due to E = 148.3022222222222, m = 14840
Overflow due to E = 148.3022222222222, m = 14841
Overflow due to E = 148.3022222222222, m = 14842
Overflow due to E = 148.3022222222222, m = 14843
Overflow due to E = 148.3022222222222, m = 14844
Overflow due to E = 148.3022222222222, m = 14845
Overflow due to E = 148.3022222222222, m = 14846
Overflow due to E = 148.3022222222222, m = 14847
Overflow due to E = 148.3022222222222, m = 14848
Overflow due to E = 

Overflow due to E = 148.3022222222222, m = 15459
Overflow due to E = 148.3022222222222, m = 15460
Overflow due to E = 148.3022222222222, m = 15461
Overflow due to E = 148.3022222222222, m = 15462
Overflow due to E = 148.3022222222222, m = 15463
Overflow due to E = 148.3022222222222, m = 15464
Overflow due to E = 148.3022222222222, m = 15465
Overflow due to E = 148.3022222222222, m = 15466
Overflow due to E = 148.3022222222222, m = 15467
Overflow due to E = 148.3022222222222, m = 15468
Overflow due to E = 148.3022222222222, m = 15469
Overflow due to E = 148.3022222222222, m = 15470
Overflow due to E = 148.3022222222222, m = 15471
Overflow due to E = 148.3022222222222, m = 15472
Overflow due to E = 148.3022222222222, m = 15473
Overflow due to E = 148.3022222222222, m = 15474
Overflow due to E = 148.3022222222222, m = 15475
Overflow due to E = 148.3022222222222, m = 15476
Overflow due to E = 148.3022222222222, m = 15477
Overflow due to E = 148.3022222222222, m = 15478
Overflow due to E = 

KeyboardInterrupt: 

In [151]:
data.reset_index(level=0, inplace=True)

In [152]:
data.to_csv('wait_prob_1.csv', index=False)

The next step is to build a time-series model for $E$.